# Leto Demo Notebook

## Install all requirements

In [2]:
! pwd

/root/Github/leto/runner_notebooks


In [3]:
cd ..

/root/Github/leto


In [4]:
! pwd

/root/Github/leto


In [5]:
!pip install aEye

In [6]:
! pip install --upgrade opencv-python

In [7]:
! pip install opencv-contrib-python

In [8]:
!apt-get update && apt-get install libgl1 -y

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1 is already the newest version (1.1.0-1).
0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.


move to root of repo to run code. 

In [9]:
! pwd

/root/Github/leto


## Imports

In [10]:
import boto3
import cv2
from cv2 import dnn_superres
import logging
from aEye import Video
from aEye import Labeler
from aEye import Aux
root = logging.getLogger()
root.setLevel(logging.INFO)
root.setLevel(logging.DEBUG)
root.setLevel(logging.ERROR)

## Create objects to process original video into the correct sizing

In [11]:
aux = Aux()

In [12]:
labeler = Labeler()

---aEye Video Label Maker v0---


# Section 2: Downsampling video

In [13]:
video_list_s3 = aux.load_s3(bucket = "leto-dish", prefix = "original-videos/benchmark/car/")
len(video_list_s3)

1

In [14]:
video_list_s3 = aux.load_s3(bucket = "leto-dish", prefix = "original-videos/benchmark/car/")
    
downsampled_video = labeler.change_resolution(video_list_s3,"360p")

aux.execute_label_and_write_local(downsampled_video)

aux.upload_s3(downsampled_video, bucket = "leto-dish", prefix ="reduced-videos/benchmark/ffmpeg-resolution-downsampler/")

aux.clean()

In [15]:
video_list_s3 = aux.load_s3(bucket = "leto-dish", prefix = "original-videos/benchmark/car/")

In [16]:
#print the metadata
print(video_list_s3[0].extract_metadata())

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'Constrained Baseline', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuv420p', 'level': 41, 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '30/1', 'avg_frame_rate': '30/1', 'time_base': '1/15360', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 153600, 'duration': '10.000000', 'bit_rate': '19960703', 'bits_per_raw_sample': '8', 'nb_frames': '300', 'extradata_size': 41, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effe

From the metadata we can see that the original codec is h.264, and the video is 1080p

In [17]:
video_list_s3 = aux.load_s3(bucket = "leto-dish", prefix = "reduced-videos/benchmark/ffmpeg-resolution-downsampler/")

In [18]:
#print the metadata
print(video_list_s3[0].extract_metadata())

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 480, 'height': 360, 'coded_width': 480, 'coded_height': 360, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'sample_aspect_ratio': '4:3', 'display_aspect_ratio': '16:9', 'pix_fmt': 'yuv420p', 'level': 30, 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '30/1', 'avg_frame_rate': '30/1', 'time_base': '1/15360', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 153600, 'duration': '10.000000', 'bit_rate': '731255', 'bits_per_raw_sample': '8', 'nb_frames': '300', 'extradata_size': 47, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impaired': 0, 'clean_effects': 0, 'attached_pic

From the metadata we can see the downsampled video's codec is h.264 and resolution 360p

# Section 2: Upsampling downsampled video using OpenCV

In [19]:
!pwd

/root/Github/leto


In [21]:
%%time 

! python reconstruction/video_upscaler.py

CPU times: user 274 ms, sys: 50 ms, total: 324 ms
Wall time: 12.5 s


# Section 2: Upsampling downsampled video using Deep Neural Networks Super Resolution 

In [22]:
%%time

! python reconstruction/video_superres.py

CPU times: user 3.22 s, sys: 354 ms, total: 3.57 s
Wall time: 2min
